In [1]:
%%configure -f
{
"conf": {"spark.serializer": "org.apache.spark.serializer.KryoSerializer",
        "spark.sql.hive.convertMetastoreParquet": "false"},

"jars": [
        "s3://emr-devel-test/jars/hudi-spark-bundle_2.11-0.5.2-incubating.jar",
        "s3://emr-devel-test/jars/spark-avro_2.11-2.4.5-amzn-0.jar"
    ]

}

In [2]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.hive.NonPartitionedExtractor
import org.apache.hudi.keygen.NonpartitionedKeyGenerator
import org.apache.hudi.keygen.SimpleKeyGenerator

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1593979559783_0010,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.hive.NonPartitionedExtractor
import org.apache.hudi.keygen.NonpartitionedKeyGenerator
import org.apache.hudi.keygen.SimpleKeyGenerator


In [3]:
val dbName = "tpcds_sparkemr_530_1gb_1_1593979210testhudi"
val warehouseName = "tpcds-warehouse-sparkemr-530-1gb-1-1593979210testhudi"
//No partitioning
/*
val hudiOptions = Map[String,String](
"hoodie.datasource.hive_sync.database" -> "tpcds_sparkemr_530_1gb_1_1593979210testhudi",
"hoodie.datasource.write.precombine.field" -> "cs_item_sk",
"hoodie.datasource.hive_sync.table" -> "catalog_sales",
"hoodie.datasource.hive_sync.enable" -> "true",
"hoodie.datasource.write.recordkey.field" -> "cs_item_sk",
"hoodie.table.name" -> "catalog_sales",
"hoodie.datasource.write.storage.type" -> "COPY_ON_WRITE",
"hoodie.datasource.write.hive_style_partitioning" -> "true",
"hoodie.parquet.max.file.size" -> String.valueOf(1024 * 1024 * 1024),
"hoodie.parquet.compression.codec" -> "snappy",
"hoodie.datasource.hive_sync.partition_extractor_class" -> "org.apache.hudi.hive.NonPartitionedExtractor",
"hoodie.datasource.hive_sync.partition_fields" -> "",
"hoodie.datasource.write.partitionpath.field" -> "",
"hoodie.datasource.write.keygenerator.class" -> "org.apache.hudi.keygen.NonpartitionedKeyGenerator"
)
*/
//Partitioning
val hudiOptions = Map[String,String](
"hoodie.datasource.hive_sync.database" -> dbName,
"hoodie.datasource.write.precombine.field" -> "cs_item_sk",
"hoodie.datasource.hive_sync.table" -> "catalog_sales",
"hoodie.datasource.hive_sync.enable" -> "true",
"hoodie.datasource.write.recordkey.field" -> "cs_item_sk",
"hoodie.table.name" -> "catalog_sales",
"hoodie.datasource.write.storage.type" -> "COPY_ON_WRITE",
"hoodie.datasource.write.hive_style_partitioning" -> "true",
"hoodie.parquet.max.file.size" -> String.valueOf(1024 * 1024 * 1024),
"hoodie.parquet.compression.codec" -> "snappy",
"hoodie.datasource.hive_sync.partition_extractor_class" -> "org.apache.hudi.hive.MultiPartKeysValueExtractor",
"hoodie.datasource.hive_sync.partition_fields" -> "cs_sold_date_sk",
"hoodie.datasource.write.partitionpath.field" -> "cs_sold_date_sk",
"hoodie.datasource.write.keygenerator.class" -> "org.apache.hudi.keygen.SimpleKeyGenerator"
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dbName: String = tpcds_sparkemr_530_1gb_1_1593979210testhudi
warehouseName: String = tpcds-warehouse-sparkemr-530-1gb-1-1593979210testhudi
hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.hive_sync.database -> tpcds_sparkemr_530_1gb_1_1593979210testhudi, hoodie.datasource.write.precombine.field -> cs_item_sk, hoodie.datasource.hive_sync.partition_fields -> cs_sold_date_sk, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.parquet.max.file.size -> 1073741824, hoodie.datasource.hive_sync.table -> catalog_sales, hoodie.parquet.compression.codec -> snappy, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> cs_item_sk, hoodie.table.name -> catalog_sales, hoodie.datasource.write.hive_style_partitioning -> true, hoodie.datasource.write.storage.type -> COPY_ON_WRITE, hoodie.datasource.write.keygener...

In [4]:
for ((k,v) <- hudiOptions) printf("%s -> %s\n", k, v)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hoodie.datasource.hive_sync.database -> tpcds_sparkemr_530_1gb_1_1593979210testhudi
hoodie.datasource.write.precombine.field -> cs_item_sk
hoodie.datasource.hive_sync.partition_fields -> cs_sold_date_sk
hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor
hoodie.parquet.max.file.size -> 1073741824
hoodie.datasource.hive_sync.table -> catalog_sales
hoodie.parquet.compression.codec -> snappy
hoodie.datasource.hive_sync.enable -> true
hoodie.datasource.write.recordkey.field -> cs_item_sk
hoodie.table.name -> catalog_sales
hoodie.datasource.write.hive_style_partitioning -> true
hoodie.datasource.write.storage.type -> COPY_ON_WRITE
hoodie.datasource.write.keygenerator.class -> org.apache.hudi.keygen.SimpleKeyGenerator
hoodie.datasource.write.partitionpath.field -> cs_sold_date_sk


In [5]:
spark.sql("use " + dbName)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res5: org.apache.spark.sql.DataFrame = []


In [6]:
spark.sql("""create external table catalog_sales_ext 
(
    cs_sold_date_sk           varchar(10)                           ,
    cs_sold_time_sk           int                           ,
    cs_ship_date_sk           int                           ,
    cs_bill_customer_sk       int                           ,
    cs_bill_cdemo_sk          int                           ,
    cs_bill_hdemo_sk          int                           ,
    cs_bill_addr_sk           int                           ,
    cs_ship_customer_sk       int                           ,
    cs_ship_cdemo_sk          int                           ,
    cs_ship_hdemo_sk          int                           ,
    cs_ship_addr_sk           int                           ,
    cs_call_center_sk         int                           ,
    cs_catalog_page_sk        int                           ,
    cs_ship_mode_sk           int                           ,
    cs_warehouse_sk           int                           ,
    cs_item_sk                int                           ,
    cs_promo_sk               int                           ,
    cs_order_number           int                           ,
    cs_quantity               int                           ,
    cs_wholesale_cost         decimal(7,2)                  ,
    cs_list_price             decimal(7,2)                  ,
    cs_sales_price            decimal(7,2)                  ,
    cs_ext_discount_amt       decimal(7,2)                  ,
    cs_ext_sales_price        decimal(7,2)                  ,
    cs_ext_wholesale_cost     decimal(7,2)                  ,
    cs_ext_list_price         decimal(7,2)                  ,
    cs_ext_tax                decimal(7,2)                  ,
    cs_coupon_amt             decimal(7,2)                  ,
    cs_ext_ship_cost          decimal(7,2)                  ,
    cs_net_paid               decimal(7,2)                  ,
    cs_net_paid_inc_tax       decimal(7,2)                  ,
    cs_net_paid_inc_ship      decimal(7,2)                  ,
    cs_net_paid_inc_ship_tax  decimal(7,2)                  ,
    cs_net_profit             decimal(7,2)                   
) 
ROW FORMAT DELIMITED FIELDS TERMINATED BY '' 
STORED AS TEXTFILE 
LOCATION 's3://tpcds-datasets/1GB/catalog_sales' """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res6: org.apache.spark.sql.DataFrame = []


In [7]:
spark.sql("select count(*) from catalog_sales_ext").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1441548]


In [8]:
val extTableDF = spark.sql("select * from catalog_sales_ext")

extTableDF.write.format("org.apache.hudi").option("hoodie.datasource.write.operation", "insert").options(hudiOptions).mode(SaveMode.Overwrite).save("s3://tpcds-warehouses-test/" + warehouseName + "/catalog_sales/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

extTableDF: org.apache.spark.sql.DataFrame = [cs_sold_date_sk: string, cs_sold_time_sk: int ... 32 more fields]


In [ ]:
//Execute the following statements in Hive beeline to fix the default partition.

//beeline -u jdbc:hive2://localhost:10000/default -n hive -p hive

//USE tpcds_sparkemr_530_1gb_1_1593979210testhudi

//ALTER TABLE catalog_sales PARTITION (cs_sold_date_sk='default') RENAME TO PARTITION (cs_sold_date_sk='0');

//ALTER TABLE catalog_sales PARTITION COLUMN (cs_sold_date_sk integer);
 
//ALTER TABLE catalog_sales PARTITION (cs_sold_date_sk='0') RENAME TO PARTITION (cs_sold_date_sk='__HIVE_DEFAULT_PARTITION__'); 

In [9]:
spark.sql("select count(*) from catalog_sales").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1439929]


In [10]:
spark.sql("select count(*) from catalog_sales where cs_sold_date_sk = 'default'").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[7180]


In [11]:
spark.sql("select count(*) from catalog_sales where cs_sold_date_sk is null").collect.foreach(println)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0]


In [12]:
spark.sql("select * from catalog_sales limit 1").printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _hoodie_commit_time: string (nullable = true)
 |-- _hoodie_commit_seqno: string (nullable = true)
 |-- _hoodie_record_key: string (nullable = true)
 |-- _hoodie_partition_path: string (nullable = true)
 |-- _hoodie_file_name: string (nullable = true)
 |-- cs_sold_time_sk: integer (nullable = true)
 |-- cs_ship_date_sk: integer (nullable = true)
 |-- cs_bill_customer_sk: integer (nullable = true)
 |-- cs_bill_cdemo_sk: integer (nullable = true)
 |-- cs_bill_hdemo_sk: integer (nullable = true)
 |-- cs_bill_addr_sk: integer (nullable = true)
 |-- cs_ship_customer_sk: integer (nullable = true)
 |-- cs_ship_cdemo_sk: integer (nullable = true)
 |-- cs_ship_hdemo_sk: integer (nullable = true)
 |-- cs_ship_addr_sk: integer (nullable = true)
 |-- cs_call_center_sk: integer (nullable = true)
 |-- cs_catalog_page_sk: integer (nullable = true)
 |-- cs_ship_mode_sk: integer (nullable = true)
 |-- cs_warehouse_sk: integer (nullable = true)
 |-- cs_item_sk: integer (nullable = true)
 |-- cs